# Custom notebook: download original and derived star photometry 

**Learning tabective:** Create 2 tables of LSST tabects: one for galaxies, one for stars

**LSST data products:** `tabect` table

**Packages:** `lsst.rsp.get_tap_service`

## 1. Import packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import lsst.geom as geom

from lsst.rsp import get_tap_service
from lsst.rsp.utils import get_pyvo_auth
from lsst.rsp.service import get_siav2_service

from rubin_sim.phot_utils import DustValues

In [ ]:
service = get_tap_service("tap")
assert service is not None

In [ ]:
# os.environ['RUBIN_SIM_DATA_DIR'] = '/rubin/rubin_sim_data'

## 2. ECDFS

### 2.1 Set target

In [ ]:
target_ra = 53.195
target_dec = -27.703
target_r = 5

We won't take into account the AGNs (present in the DIAtabect catalog). We will only look at galaxies and stars in the tabect catalog. To differentiate them efficiently within the LSST catalog, we establish a selection criterion : if 3 bands or more show {band}_extendedness = 1, we'll consider it as a galaxies. Unless the sources will be flagged as stars. But that something that will be useful later.

### 2.2 Query

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, ebv, " + \
        "u_psfMag, u_psfMagErr, " + \
        "g_psfMag, g_psfMagErr, " + \
        "r_psfMag, r_psfMagErr, " + \
        "i_psfMag, i_psfMagErr, " + \
        "z_psfMag, z_psfMagErr, " + \
        "y_psfMag, y_psfMagErr " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (r_psfFlux/r_psfFluxErr > 10) AND " + \
        "(shape_flag = 0) AND " + \
        "(g_extendedness = 0) AND " + \
        "(r_extendedness = 0) AND " + \
        "(i_extendedness = 0) AND " + \
        "(z_extendedness = 0) AND " + \
        "CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS',"+str(target_ra)+","+str(target_dec)+", "+str(target_r)+")) = 1 "



In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
tab = results.to_table()
tab

### 2.3 Apply dust correction and SDSS color conversion

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/rubin/rubin_sim_data'

In [ ]:
bands = "ugri"
plt.rcParams["font.size"] = 15
plt.rcParams["figure.titlesize"] = 15
plt.rcParams["legend.fontsize"] = 12
plt.rcParams["axes.titlesize"] = 12

Apply dust correction

In [ ]:
R_band = DustValues().r_x

In [ ]:
A_band = {band: R_band[band] * tab['ebv'] for band in bands}

In [ ]:
for band in bands:
    tab[f"{band}_psfMag0"] = tab[f"{band}_psfMag"] - A_band[band]

Transform the dust-corrected photometry to the SDSS system, following the equations from [RTN-099](https://rtn-099.lsst.io/).

In [ ]:
tab["ug"] = tab["u_psfMag0"] - tab["g_psfMag0"]
tab["gr"] = tab["g_psfMag0"] - tab["r_psfMag0"]
tab["gi"] = tab["g_psfMag0"] - tab["i_psfMag0"]
tab["ri"] = tab["r_psfMag0"] - tab["i_psfMag0"]

tab["gi_sdss"] = 1.065*tab["gi"] + 0.005
tab["gr_sdss"] = 1.058*tab["gr"] + 0.058*tab["gr"] - 0.002
tab["u_sdss"] = tab["u_psfMag0"] + 0.063*tab["gi"] **2 - 0.192*tab["gi"] + 0.263

In [ ]:
g_cri = (tab["g_psfMag0"] > 14) & (tab["g_psfMag0"] < 24)
gr_cri = (tab["gr"] > 0) & (tab["gr"] < 1)
ug_gr_cri = ((tab["ug"] > 0.5) & (tab["ug"] < 2.0) &
             (tab["gr"] - 0.5 * tab["ug"] > -0.5) & (tab["gr"] - 0.5 * tab["ug"] < 0.1))
gr_ri_cri = (0.35*tab["gr"] - tab["ri"] > -0.5) & (0.35*tab["gr"] - tab["ri"] < 0.5)

In [ ]:
sel = (A_band["r"] < 0.5) & g_cri & gr_cri & ug_gr_cri & gr_ri_cri
print(f"There are {len(tab[sel])} stars suitable for this analysis.")

### 2.4 Derive stellar parameters (Teff and [Fe/H])

Teff

In [ ]:
from astropy.table import MaskedColumn

tab['log_teff'] = MaskedColumn(np.zeros(len(tab)), mask=np.ones(len(tab), dtype=bool))

tab['log_teff'][sel] = 3.872 - 0.264 * tab['gr_sdss'][sel]
tab['log_teff'].mask[sel] = False

In [ ]:
plt.scatter(np.ma.filled(tab["ug"], np.nan), np.ma.filled(tab["gr"], np.nan),
            c=10**np.ma.filled(tab["log_teff"], np.nan), cmap=plt.cm.jet_r)
plt.xlabel('u-g')
plt.ylabel('g-r')
cb = plt.colorbar()
cb.set_label(r'$T_{eff}$ [K]')

[Fe/H]

In [ ]:
def feh_phot_ivezic2008(u_minus_g, g_minus_r):
    """
    Compute photometric metallicity [Fe/H]_ph from SDSS colors using the
    calibration of Ivezić et al. (2008, ApJ, 684, 287; Eq. 4).

    This empirical relation estimates stellar metallicity for F/G-type
    main-sequence stars based on their (u–g) and (g–r) colors.

    Parameters
    ----------
    u_minus_g : array_like
        The (u–g) color(s) of the star(s).
    g_minus_r : array_like
        The (g–r) color(s) of the star(s).

    Returns
    -------
    feh : ndarray
        Photometric metallicity [Fe/H]_ph, in dex.
    """
    ug = np.asarray(u_minus_g, dtype=float)
    gr = np.asarray(g_minus_r, dtype=float)

    x = np.where(gr <= 0.4, ug, ug - 2.0*gr + 0.8)
    y = gr

    a, b, c, d, e, f, g, h, i, j = (
        -4.37, -8.56, 15.5, -39.0, 23.5, 20.5, 12.1, 7.33, -10.1, -21.4
    )

    feh = (a + b*x + c*y + d*x*y + e*x**2 + f*y**2
             + g*x**2*y + h*x*y**2 + i*x**3 + j*y**3)

    return feh

In [ ]:
tab['feh'] = MaskedColumn(np.zeros(len(tab)), mask=np.ones(len(tab), dtype=bool))
tab['feh'][sel] = feh_phot_ivezic2008(tab['ug'][sel], tab['gr'][sel])
tab['feh'].mask[sel] = False
fehtest = feh_phot_ivezic2008(tab['ug'][sel], tab['gr'][sel])
tab['feh'][sel] 

In [ ]:
plt.scatter(tab["ug"], tab["gr"], c=tab['feh'], cmap=plt.cm.jet)
plt.xlabel('u-g')
plt.ylabel('g-r')
cb = plt.colorbar()
cb.set_label('[Fe/H]')

### 2.5 Save resulting table with derived columns

Save table

In [ ]:
tab.write('DP1_ECDFS_star_photometry.csv', format='csv', overwrite=True)


In [ ]:
job.delete()
del query, results, tab

### 2.6 Including galaxies

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, ebv, " + \
        "u_cModelMag, u_cModelMagErr, " + \
        "g_cModelMag, g_cModelMagErr, " + \
        "r_cModelMag, r_cModelMagErr, " + \
        "i_cModelMag, i_cModelMagErr, " + \
        "z_cModelMag, z_cModelMagErr, " + \
        "y_cModelMag, y_cModelMagErr, " + \
        "u_psfMag, u_psfMagErr, " + \
        "g_psfMag, g_psfMagErr, " + \
        "r_psfMag, r_psfMagErr, " + \
        "i_psfMag, i_psfMagErr, " + \
        "z_psfMag, z_psfMagErr, " + \
        "y_psfMag, y_psfMagErr, " + \
        "u_Extendedness " + \
        "g_Extendedness " + \
        "r_Extendedness " + \
        "i_Extendedness " + \
        "z_Extendedness " + \
        "y_Extendedness " + \
        "refExtendedness " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (r_cModelFlux/r_cModelFluxErr > 1) AND " + \
        "(shape_flag = 0) AND " + \
        "CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS',"+str(target_ra)+","+str(target_dec)+", "+str(target_r)+")) = 1 "



In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
tab = results.to_table()

In [ ]:
tab.write('DP1_ECDFS_photometry.csv', format='csv', overwrite=True)

In [ ]:
job.delete()
del query, results, tab